In [3]:
import pandas as pd
import numpy as np

df_raw = pd.read_csv("data/micro_world.csv", encoding='latin1', on_bad_lines='skip')

df_raw.head()

,economy,economycode,regionwb,pop_adult,wpid_random,wgt,female,age,educ,inc_q,...,receive_transfers,receive_pension,receive_agriculture,pay_utilities,remittances,mobileowner,internetaccess,anydigpayment,merchantpay_dig,year
0,Afghanistan,AFG,South Asia,22647496.0,144274031,0.716416,2,43.0,2,4,...,4,4,4.0,1,5.0,1,2,1,0.0,2021
1,Afghanistan,AFG,South Asia,22647496.0,180724554,0.497408,2,55.0,1,3,...,4,4,2.0,4,5.0,1,2,0,0.0,2021
2,Afghanistan,AFG,South Asia,22647496.0,130686682,0.650431,1,15.0,1,2,...,4,4,4.0,4,3.0,2,2,0,0.0,2021
3,Afghanistan,AFG,South Asia,22647496.0,142646649,0.991862,2,23.0,1,4,...,4,4,2.0,4,5.0,1,2,0,0.0,2021
4,Afghanistan,AFG,South Asia,22647496.0,199055310,0.554940,1,46.0,1,1,...,4,4,4.0,4,5.0,2,2,0,0.0,2021


In [14]:

# columns to keep 
identifier_cols = [
    "economy",        # country name
    "regionwb",       # region (e.g., SSA, South Asia)
]

demographic_cols = [
    "female",         # 1=female, 2=male
    "age",            
    "educ",           # education level
    "urbanicity",     # urban/rural
    "inc_q",          # income quintile
    "emp_in",
    "year"         # employment status (your “employment” variable)

]


finclusion_cols = [
    "account",            # has any account
    "account_fin",        # bank account
    "account_mob",        # mobile money account
    "dig_account",        # digital-ready account
    "anydigpayment"       # used digital payments
]

entrepreneur_cols = [
    "fin17a",   # saved at a financial institution
    "fin17b",   # saved using mobile money
    "fin18",    # saved any money in past year
    "fin20",    # applied for a loan using a phone
    "fin21",    # received a mobile loan
    "fin22e"    # borrowed to start / operate a business
]



barrier_cols = [
    "fin11a",   # too far
    "fin11b",   # too expensive
    "fin11c",   # lack documentation
    "fin11d",   # lack money
    "fin11f",   # lack trust
    "fin14a",   # mobile money: not knowing how
    "fin14b",   # mobile money: too expensive
    "fin14c",   # mobile money: no agent nearby
    "fin14d",   # mobile money: no phone
    "fin14e"    # mobile money: don’t trust
]

digital_cols = [
    "con30a","con30b","con30d","con30e","con30f","con30h",
    "con31a","con31b","con31c","con31d","con31e","con31f","con31g","con31h"
]


payment_behavior_cols = [
    "borrowed",
    "saved",
    "receive_wages",
    "receive_transfers",
    "receive_pensions",
    "receive_agriculture",
    "merchantpay_dig",
    "pay_utilities",
    "domestic_remittances",
    "anydigpayment"
]


resilience_cols = [
    "fin28",   # ability to come up with emergency funds
    "fin29",   # time needed to come up with funds
    "fin30",   # financial difficulty
    "fin31a","fin31b","fin31c","fin31d","fin31e","fin31f","fin31g","fin31h",
    "fin32"    # coping strategies
]

stress_cols = [
    "fin33",   # financial stress level
    "fin34a","fin34b","fin34c","fin34d",  # difficulty paying things
    "fin35",
    "fin36","fin36a",
    "fin37","fin38",
    "fin39a","fin39b","fin39c","fin39d",
    "fin40","fin41","fin41a",
    "fin42"
]

literacy_cols = [
    "fin43a","fin43b","fin43c","fin43d",   # numeracy & risk questions
    "fin44",                               # knowledge of interest rate
    "fin45"                                # understanding compounding
]

payments_extended_cols = [
    "con2a","con2b","con2c","con2d","con2e","con2f","con2g",
    "con5","con6","con7","con8"
]

save_borrow_cols = [
    "fin13_1","fin13a","fin13b","fin13c","fin13d","fin13e","fin13f",
    "fin24","fin24a","fin24b","fin24c","fin24d1","fin24d2","fin24d3",
    "fin25e1","fin25e2","fin25e3","fin25e4"
]






columns_to_keep = (
    identifier_cols +
    demographic_cols +
    finclusion_cols +
    entrepreneur_cols +
    barrier_cols +
    digital_cols +
    payment_behavior_cols +
    resilience_cols+
    stress_cols +
    literacy_cols +
    payments_extended_cols +
    save_borrow_cols
)




In [15]:
# SAFELY SELECT 

available_cols = [c for c in columns_to_keep if c in df_raw.columns]
df = df_raw[available_cols].copy()

print("Using", len(available_cols), "columns.")

Using 59 columns.


In [16]:
# clean basic codes
import numpy as np
df = df.replace([".."," ","","NAN","N/A",998,999], np.nan)

In [17]:
# clean employment

if "emp_in" in df.columns:
    employment_map = {
        1: "In Labor Force",
        2: "Not in Labor Force",
        98: "Don't Know",
        99: "Refused"
    }
    df["employment_status"] = df["emp_in"].map(employment_map)
    df.drop(columns=["emp_in"], inplace=True)

In [18]:
# clean urbanicity
if "urbanicity" in df.columns:
    df["urban_rural"] = df["urbanicity"].map({
        1: "Urban",
        2: "Rural"
    })
    df.drop(columns=["urbanicity"], inplace=True)


In [19]:
# clean urbanicity
if "urbanicity" in df.columns:
    df["urban_rural"] = df["urbanicity"].map({
        1: "Urban",
        2: "Rural"
    })
    df.drop(columns=["urbanicity"], inplace=True)


In [20]:
# clean simple yes or no 
yes_no_vars = [
    "account","account_fin","account_mob","dig_account",
    "anydigpayment","borrowed","saved","merchantpay_dig"
]

def safe_yes_no(value):
    try:
        # Try to coerce to float
        v = float(value)
        if v == 1:
            return "Yes"
        if v == 0:
            return "No"
        if v in [98, 99]:
            return None
    except:
        return None
    return None

for col in yes_no_vars:
    if col in df.columns:
        df[col] = df[col].apply(safe_yes_no)



/var/folders/jj/rpbl4z5j3bj21p6gz5f128c80000gn/T/ipykernel_31897/1585409924.py:23: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df[col] = df[col].apply(safe_yes_no)


In [21]:
multi_category_vars = [
    "receive_wages","receive_transfers","receive_pensions",
    "receive_agriculture","domestic_remittances","pay_utilities"
]

multi_map = {
    1: "Into Account / Digital",
    2: "Cash Only",
    3: "Other Method",
    4: "Did Not Receive / Not Applicable",
    5: "Don't Know / Refused"
}

def safe_multi(value):
    try:
        v = int(float(value))
        return multi_map.get(v, None)
    except:
        return None

for col in multi_category_vars:
    if col in df.columns:
        df[col] = df[col].apply(safe_multi)


In [22]:
#clean gender
if "female" in df.columns:
    df["gender"] = df["female"].map({1: "Female", 2: "Male"})
    df.drop(columns=["female"], inplace=True)


In [23]:
# Financial Resilience Cleaning
resilience_map = {
    1: "Yes",
    2: "No",
    3: "DK",
    4: "Refused",
    98: None,
    99: None
}

for col in resilience_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].map(resilience_map)


In [24]:
# Financial Stress Cleaning
stress_map = {
    1: "Yes",
    2: "No",
    3: "DK",
    4: "Refused",
    98: None,
    99: None
}

for col in stress_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].map(stress_map)


In [25]:
digital_activity_map = {
    1: "Yes",
    2: "No",
    98: None,
    99: None
}

for col in digital_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].map(digital_activity_map)


In [26]:
payments_map = {
    1: "Yes",
    2: "No",
    3: "DK",
    4: "Refused",
    98: None,
    99: None
}

for col in payments_extended_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].map(payments_map)


In [27]:
save_borrow_map = {
    1: "Yes",
    2: "No",
    3: "DK",
    4: "Refused",
    98: None,
    99: None
}

for col in save_borrow_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")
        df[col] = df[col].map(save_borrow_map)


In [28]:
import openpyxl

df.to_csv("tableau/findex_2021_clean_.csv", index=False)


In [29]:
# -----------------------------------------------------------
# 3. RENAME COLUMN DICTIONARY 
# -----------------------------------------------------------

rename_dict = {

    # IDENTIFIERS
    "economy": "Country",
    "regionwb": "Region",
    "year": "Year",

    # DEMOGRAPHICS
    "female": "Gender",
    "age": "Age",
    "educ": "Education Level",
    "urbanicity": "Urban/Rural",
    "inc_q": "Income Quintile",
    "emp_in": "Employment Status",

    # FINANCIAL INCLUSION
    "account": "Has Account",
    "account_fin": "Has Bank Account",
    "account_mob": "Has Mobile Money Account",
    "dig_account": "Has Digital Account",
    "anydigpayment": "Made Any Digital Payment",

    # ENTREPRENEURSHIP
    "fin17a": "Saved At Financial Institution",
    "fin17b": "Saved Using Mobile Money",
    "fin17c": "Saved Using Savings Club",
    "fin17d": "Saved At Home",
    "fin17e": "Saved For Emergencies",
    "fin17f": "Saved For Business",
    "fin18": "Saved Any Money Last Year",
    "fin20": "Applied For Loan By Phone",
    "fin21": "Received Mobile Loan",
    "fin22a": "Borrowed For Health",
    "fin22b": "Borrowed For Education",
    "fin22c": "Borrowed For Food",
    "fin22d": "Borrowed For Emergencies",
    "fin22e": "Borrowed For Business",
    "fin22f": "Borrowed For Farming",
    "fin22g": "Borrowed For Housing",
    "fin22h": "Borrowed For Other",

    # BARRIERS
    "fin11a": "Barrier_Distance",
    "fin11b": "Barrier_Too_Expensive",
    "fin11c": "Barrier_No_Documentation",
    "fin11d": "Barrier_No_Money",
    "fin11f": "Barrier_No_Trust",
    "fin14a": "MM_Barrier_No_Knowledge",
    "fin14b": "MM_Barrier_Too_Expensive",
    "fin14c": "MM_Barrier_No_Agent",
    "fin14d": "MM_Barrier_No_Phone",
    "fin14e": "MM_Barrier_No_Trust",

    # DIGITAL ACCESS
    "internet_use": "Used Internet",
    "con1": "Owns Mobile Phone",
    "con12": "Phone Use Frequency",
    "con25": "Used Internet Last 3 Months",

    # DIGITAL ACTIVITY (CON30 & CON31)
    "con30a": "Online_Government_Services",
    "con30b": "Online_Shopping",
    "con30c": "Uses_Social_Media",
    "con30d": "Online_Education",
    "con30e": "Online_Banking",
    "con30f": "Online_Loan_Applications",
    "con30g": "Used_Govt_Services_Online",
    "con30h": "Online_Healthcare_Services",

    "con31a": "Used_Phone_For_Payments",
    "con31b": "Used_Phone_For_Remittances",
    "con31c": "Used_Phone_For_Savings",
    "con31d": "Used_Phone_For_Investments",
    "con31e": "Used_Phone_For_Banking",
    "con31f": "Used_Phone_For_Credit",
    "con31g": "Used_Phone_For_Insurance",
    "con31h": "Used_Phone_For_Financial_Management",

    # PAYMENT BEHAVIOR
    "borrowed": "Borrowed Last Year",
    "saved": "Saved Last Year",
    "receive_wages": "Received Wages",
    "receive_transfers": "Received Government Transfers",
    "receive_pensions": "Received Pension",
    "receive_agriculture": "Received Agricultural Payments",
    "merchantpay_dig": "Made Digital Merchant Payment",
    "pay_utilities": "Paid Utility Bills",
    "domestic_remittances": "Domestic Remittances",

    # PAYMENTS EXTENDED
    "con2a": "Paid_Electricity",
    "con2b": "Paid_Water",
    "con2c": "Paid_Internet",
    "con2d": "Paid_TV",
    "con2e": "Paid_School_Fees",
    "con2f": "Paid_Rent",
    "con2g": "Paid_Health_Services",
    "con5": "Used_Card_To_Pay",
    "con6": "Used_Cash_To_Pay",
    "con7": "Used_Mobile_Money_To_Pay",
    "con8": "Used_Bank_Transfer_To_Pay",

    # FINANCIAL RESILIENCE
    "fin28": "Emergency_Funds_Available",
    "fin29": "Time_To_Raise_Emergency_Funds",
    "fin30": "Financial_Difficulty_Level",
    "fin31a": "Coped_Cut_Spending",
    "fin31b": "Coped_Borrowed_Money",
    "fin31c": "Coped_Used_Savings",
    "fin31d": "Coped_Sold_Assets",
    "fin31e": "Coped_Financial_Assistance",
    "fin31f": "Coped_Extra_Work",
    "fin31g": "Coped_Govt_Assistance",
    "fin31h": "Coped_Other",
    "fin32": "Main_Coping_Strategy",

    # FINANCIAL STRESS
    "fin33": "Stress_Level",
    "fin34a": "Difficulty_Paying_Rent",
    "fin34b": "Difficulty_Paying_School_Fees",
    "fin34c": "Difficulty_Paying_Bills",
    "fin34d": "Difficulty_Buying_Food",
    "fin35": "Had_Unexpected_Expenses",
    "fin36": "Unable_To_Meet_Basic_Needs",
    "fin36a": "Missed_Payments",
    "fin37": "High_Debt_Burden",
    "fin38": "Reduced_Essential_Spending",
    "fin39a": "Borrowed_From_Family",
    "fin39b": "Borrowed_From_Friends",
    "fin39c": "Borrowed_From_Bank",
    "fin39d": "Borrowed_From_MM",
    "fin40": "Sold_Assets",
    "fin41": "Late_On_Rent",
    "fin41a": "Late_On_Utilities",
    "fin42": "Financial_Shock_Experienced",

    # FINANCIAL LITERACY
    "fin43a": "Correct_Interest_Comparison",
    "fin43b": "Understands_Compound_Interest",
    "fin43c": "Understands_Risk",
    "fin43d": "Understands_Inflation",
    "fin44": "Understands_Interest_Rate",
    "fin45": "Understands_Financial_Terms",

    # SAVING & BORROWING PURPOSES
    "fin13_1": "Saved_For_Education",
    "fin13a": "Saved_For_Health",
    "fin13b": "Saved_For_Housing",
    "fin13c": "Saved_For_Business",
    "fin13d": "Saved_For_Wedding",
    "fin13e": "Saved_For_Funeral",
    "fin13f": "Saved_For_Emergencies",
    "fin24": "Borrowed_Purpose",
    "fin24a": "Borrowed_For_Education",
    "fin24b": "Borrowed_For_Health",
    "fin24c": "Borrowed_For_Business",
    "fin24d1": "Borrowed_For_Food",
    "fin24d2": "Borrowed_For_Housing",
    "fin24d3": "Borrowed_For_Emergencies",
    "fin25e1": "Loan_For_School_Fees",
    "fin25e2": "Loan_For_Medical",
    "fin25e3": "Loan_For_Business",
    "fin25e4": "Loan_For_Agro",
}

# -----------------------------------------------------------
# 4. APPLY RENAMING
# -----------------------------------------------------------

df = df.rename(columns=rename_dict)


In [30]:
df.to_csv("tableau/findex_2021_tableau_ready.csv", index=False)

print("✔ Tableau-ready dataset saved: tableau/findex_2021_tableau_ready.csv")

✔ Tableau-ready dataset saved: tableau/findex_2021_tableau_ready.csv
